In [4]:
import json

fp = "/Users/lucas/Downloads/r3_e2e-2022-09-13_18-23.log"

with open(fp, "r") as f:
    lines = [json.loads(ln) for ln in f]


In [5]:
grouped = {}
for ln in lines:
    backbone = ln['config']['backbone']
    precision = 'fp16' if ln['config']['cache']['half_precision'] else 'fp32'
    for key, val in ln['report'].items():
        if not isinstance(val, float):
            continue
        grouped.setdefault(f"{backbone}_{precision}", {}).setdefault(key, []).append(val)

for key, val in grouped.items():
    for k, v in val.items():
        grouped[key][k] = sum(v) / len(v)

print(grouped)

{'nreimers/MiniLMv2-L6-H384-distilled-from-BERT-Large_fp16': {'caching_on': 0.06835995035723222, 'caching_off': 0.050786251066613976, 'caching_prefetch': 0.027007407534080246, 'caching_use': 0.02410580116386068}, 'nreimers/MiniLMv2-L6-H384-distilled-from-BERT-Large_fp32': {'caching_on': 0.07619102195415474, 'caching_off': 0.050644269847156774, 'caching_prefetch': 0.02602600891821875, 'caching_use': 0.024717729047544826}, 'nreimers/MiniLMv2-L6-H768-distilled-from-BERT-Large_fp16': {'caching_on': 0.07389322982346716, 'caching_off': 0.0898419180895953, 'caching_prefetch': 0.04531704258233813, 'caching_use': 0.0451325759414799}, 'nreimers/MiniLMv2-L6-H768-distilled-from-BERT-Large_fp32': {'caching_on': 0.10098466569164145, 'caching_off': 0.0898440197573455, 'caching_prefetch': 0.04489953824733477, 'caching_use': 0.04655871143588772}, 'bert-base-uncased_fp16': {'caching_on': 0.06903840490609162, 'caching_off': 0.1735499995470704, 'caching_prefetch': 0.08590737603302782, 'caching_use': 0.086

In [7]:
NUM_EPOCHS = 6
for system, report in grouped.items():
    if 'caching_off' not in report:
        continue
    print(f'--- {system} ---')
    no_caching = report['caching_off']
    storing_and_caching = (
        report['caching_on'] + report['caching_use'] * (NUM_EPOCHS - 1)
    ) / NUM_EPOCHS
    just_caching = report['caching_use']
    print(f"no caching: {no_caching:.4f}")
    print(f"storing and caching: {storing_and_caching:.4f}")
    print(f"just caching: {just_caching:.4f}")
    print('\n\n')


--- nreimers/MiniLMv2-L6-H384-distilled-from-BERT-Large_fp16 ---
no caching: 0.0508
storing and caching: 0.0315
just caching: 0.0241



--- nreimers/MiniLMv2-L6-H384-distilled-from-BERT-Large_fp32 ---
no caching: 0.0506
storing and caching: 0.0333
just caching: 0.0247



--- nreimers/MiniLMv2-L6-H768-distilled-from-BERT-Large_fp16 ---
no caching: 0.0898
storing and caching: 0.0499
just caching: 0.0451



--- nreimers/MiniLMv2-L6-H768-distilled-from-BERT-Large_fp32 ---
no caching: 0.0898
storing and caching: 0.0556
just caching: 0.0466



--- bert-base-uncased_fp16 ---
no caching: 0.1735
storing and caching: 0.0839
just caching: 0.0869



--- bert-base-uncased_fp32 ---
no caching: 0.1735
storing and caching: 0.0903
just caching: 0.0882



